In [ ]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

In [1]:
print(1)

1


In [2]:
from labjack import ljm

<class 'labjack.ljm.ljm.LJMError'>: Cannot load the LJM library libLabJackM.so. libLabJackM.so: cannot open shared object file: No such file or directory


In [3]:
!echo $LD_LIBRARY_PATH

/home/craiglagegit/auto-op-env-packages/atmospec/lib:/scratch/opt/OpenSpliceDDS/V6.10.4/HDE/x86_64.linux/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/astshim/g545b398f03+ce83317b7e/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/log/gd27fc4addd+43e67d6499/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/sphgeom/gfb2c8f9998+ce83317b7e/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/geom/gfd8e8de604+8836aef69c/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/pex_config/gc9a196ae7d+971de38e4f/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/pex_exceptions/g958458c828+fc3c6d1a77/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/base/gc57f72a14a+ce83317b7e/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/cpputils/g1ffd6c4751+50c04d9c73/lib:/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-2.0.0/Linux64/daf_bas

In [4]:
!ls /home/craiglagegit/.local/lib

libLabJackM.so.1.20.1  python3.8


In [ ]:
# Set Cerro Pachon location
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
#Start classes
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

In [ ]:
# enable components
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable()

In [ ]:
# ATArchiver didn't come up
tmp=await latiss.rem.atarchiver.cmd_start.set_start()
print(tmp)

In [ ]:
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.ENABLED)

In [ ]:
await latiss.rem.atarchiver.cmd_enable.set_start(timeout=10)

In [ ]:
# These 8 cells were run earlier to install the labJack in a box

In [ ]:
# Lowering to check box clearance
start_az = 0.0
start_el = 20.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

In [ ]:
# Failed.  Bring back up
start_az = 0.0
start_el = 80.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# Testing new location
start_az = 0.0
start_el = 20.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# Looks OK.  Bring back up
start_az = 0.0
start_el = 80.0
start_rot = 0.0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot)

In [ ]:
# Running more tests

In [ ]:
await latiss.take_bias(1)

In [ ]:
# Take 50 biases seq # 5-54
# Added wait to stop killing the recent images
for i in range(50):
    await asyncio.sleep(2.0)
    await latiss.take_bias(1)

In [ ]:
# Re-aligning flat illuminator
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.prepare_for_flatfield()

In [ ]:
await atcs.stop_tracking()

In [ ]:
await atcs.shutdown()

In [ ]:
await latiss.standby()